In [6]:
from openai import OpenAI

def read_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content.rstrip('\n')


def openai_chat(messages):
    client = OpenAI(api_key=read_file("/home/teveritt/OpenAI-API-DCU-AI.key"))

    return client.chat.completions.create(
        model='gpt-4o',
        temperature=0.0,
        messages=messages,
        stream=False,
        max_tokens=1525
    )

In [19]:
def fix_gdpr(text):
    m = [
        {
            "role": "system",
            "content": """
This GPT will anonymise job postings by removing and replacing personally identifiable information (PII) with specific placeholders. Replace the following PII with the given placeholders:

    - Person's name (Not Company/Employer): Replace with "[Name Redacted]"
    - Addresses: Replace with "[Address Redacted]"
    - Phone Numbers: Replace with "[Phone Number Redacted]"
    - Phone Number Extension: Replace with "[Extension Redacted]"
    - Email Addresses: Replace with "[Email Redacted]"
    - IP Addresses: Replace with "[IP Address Redacted]"
    - URL: Replace with "[URL Redacted]"

Ensure that all PII is thoroughly and consistently anonymised. Here are some examples:

    - Original: "We are seeking a Software Engineer. Please contact John Doe at john.doe@example.com. Our office is located at 1234 Elm Street, London. For more information, call +44 1234 567890 ext. 1234. Visit us online at ourdomain.com from IP 192.168.1.1."
    - Anonymised: "We are seeking a Software Engineer. Please contact [Name Redacted] at [Email Redacted]. Our office is located at [Address Redacted]. For more information, call [Phone Number Redacted] ext. [Extension Redacted]. Visit us online at [URL Redacted] from [IP Address Redacted]. "

Apply these replacements to any job posting provided and don't alter anything else.
If there is nothing to anonymise, then respond with <SKIP>.
"""
        }, {
            "role": "user",
            "content": text
        }
    ]
    output = openai_chat(m)
    content = output.choices[0].message.content.strip()
    content = content.replace('***', '')
    content = content.replace('**', '')
    content = content.replace('*', '-')
    return content

In [20]:
import os

with open('processed-gdpr.txt', 'r') as file:
    processed = set()
    for line in file.readlines():
        processed.add(line.strip().replace('\n',''))

processed = list(processed)
processed.sort()
print(f'Processed: {processed}')

for type in ['train','val', 'test']:
    dataframes = []
    for file_name in os.listdir(type):
        file_name, file_extension = os.path.splitext(file_name)
        base_path = os.path.join(type, file_name)
        if f'{base_path}.txt' in processed:
            print(f'Already processed {base_path}.txt')
        else:
            if file_name.startswith('Kaggle'):
                if os.path.isfile(f'{base_path}.txt'):
                        print(f'Processing {base_path}.txt')
                        with open(f'{base_path}.txt', 'r') as file:
                            text = file.read().strip()
                            text = fix_gdpr(text)
                        if text == '<SKIP>':
                            print(f'Skipping {base_path}.txt')
                        else:
                            print(f'Cleaning {base_path}.txt')
                            with open(f'{base_path}.txt', 'w') as f:
                                f.write(text)
                            
                        processed.append(f'{base_path}.txt')
                        with open('processed-gdpr.txt', 'w') as file:
                            for item in processed:
                                file.write('%s\n' % item)
                            
        #break
    #break
                
